In [92]:
# install of utilities to display openai environment in video
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils  > /dev/null 2>&1

In [93]:
!pip install pct


In [94]:
import time
import gym
import math
import pct.putils as vid

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [95]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from pct.environments import MountainCarContinuousV0

from pct.architectures import LevelKey
from pct.architectures import DynamicArchitecture
from pct.structure import ArchitectureStructure


In [96]:
verbose=False
seed=66
render=True#False#True
runs=200
inputs =  [0, 1]
references =  [0.45]
top_inputs =  [0]
error_collector_type = 'ReferencedInputsError'
error_response_type = 'RootMeanSquareError'
error_properties = [['referenced_inputs','0&0.45']]
error_limit = 10
inputs_names = ['IP', 'IV']
loops = 1
history=True
seed = 66
draw=True
plot=True

In [97]:
arch_structure = None
modes =  {LevelKey.ZERO:6, LevelKey.N:6,LevelKey.TOP:5,LevelKey.ZEROTOP :5}
arch_structure = ArchitectureStructure(modes=modes)

In [98]:
# raws
raw1= [[[[0.20537720677029642, -0.07542428891574429, -0.5753709315879731]], [[0.5125002413720094, 0.9030642857490316], [-0.8353291651612907, 0.8297959913400866], [-0.4288756555796329, 0.07659651246475496]], [[0.8961438486249399], [1.1022272130518322], [0.9726460281234541]], [[1.0360708077917133, -0.8168549347160642, -0.4004103782010071]]], [[[1.0]], [[0.9268938948943133, 0.16642593250792734]], [0.45]]]
raw2=[[[[0.20537720677029642, -0.07542428891574429, -0.5753709315879731]], [[0.5125002413720094, 0.9030642857490316], [-0.8353291651612909, 0.8297959913400865], [-0.4288756555796329, 0.07659651246475496]], [[0.8961438486249399], [1.1022272130518322], [0.9726460281234541]], [[1.0360708077917133, -0.8168549347160642, -0.4004103782010071]]], [[[1.0]], [[0.9268938948943133, 0.16642593250792734]], [0.45]]]

raw3=[[[[0.3289677707715672, 0.3339450991874028, 0.47349153623015794]], [[1.9076389941199927, 0.8774878555385002], [-0.9355588847432326, 0.8699646050543319], [0.7541640000027829, 0.24271135919664683]], [[1.1674647376347207], [1.7644123853538156], [2.2808148489927476]], [[2.1156006247695602, 1.8679623529253222, -0.7024276093372841]]], [[[1.3130235832953403]], [[1.2187673638776089, 0.3992076673249729]], [0.45]]]

raw4=[[[[1.0190259303271414, 1.2913100401222737, -0.546837393643841]], [[3.3507493791629113, 0.9425521605494227], [0.3789739983757472, 0.91172731902103], [1.7595560023375687, 0.2835552402223259]], [[0.6704990351863104], [1.4329353833290213], [3.142389571048783]], [[1.8225673149478034, 3.6876120643549894, -0.9068435846113589]]], [[[0.06989072428946996]], [[2.0901703235098035, 0.38214051658164266]], [0.45]]]

raw5=[[[[1.0190259303271414, 1.2913100401222737, -0.546837393643841]], [[3.3507493791629113, 0.9425521605494227], [0.3789739983757472, 0.91172731902103], [1.7595560023375687, 0.2835552402223259]], [[0.6704990351863103], [1.4329353833290213], [3.1423895710487826]], [[1.8225673149478034, 3.6876120643549894, -0.9068435846113589]]], [[[0.06989072428946996]], [[2.0901703235098035, 0.3821405165816426]], [0.45]]]

raw6=[[[[1.4567102760152006, 0.50067180601942, 1.4148187877238658]], [[2.9720167466648344, 0.9397939212548161], [-0.25094384189300306, 0.8277886361674924], [2.478243299372794, 0.24641774996682608]], [[0.657839205963882], [2.3224654938578], [2.9998272743058703]], [[3.098325308726217, 1.7275367998871547, -0.3367756467994916]]], [[[0.2834445323942473]], [[2.0006502040311576, 0.38974879004744445]], [0.45]]]

raw7=[[[[1.4567102760152006, 0.50067180601942, 1.4148187877238658]], [[2.9720167466648344, 0.9397939212548161], [-0.25094384189300306, 0.8277886361674924], [2.478243299372794, 0.24641774996682608]], [[0.657839205963882], [2.3224654938578], [2.9998272743058703]], [[3.098325308726217, 1.7275367998871547, -0.3367756467994916]]], [[[0.2834445323942473]], [[2.0006502040311576, 0.38974879004744445]], [0.45]]]

raw8=[[[[1.198998468541923, 0.6117542042667189, 1.7677170617308327]], [[3.434111762493844, 0.9640180391471003], [-0.05887913748075457, 0.9825724309247841], [2.5730535955638225, 0.38627004681749383]], [[1.137932637787607], [2.1164923802567945], [3.232824945210956]], [[3.0946849052258356, 3.1217341494218322, -0.4583645532177818]]], [[[0.5030156034214709]], [[3.9129702537620084, 0.4591226113143575]], [0.45]]]

raw9=[[[[1.198998468541923, 0.6117542042667189, 1.7677170617308327]], [[3.434111762493844, 0.9640180391471003], [-0.05887913748075457, 0.9825724309247841], [2.5730535955638225, 0.38627004681749383]], [[1.137932637787607], [2.1164923802567945], [3.232824945210956]], [[3.0946849052258356, 3.1217341494218322, -0.4583645532177818]]], [[[0.5030156034214709]], [[3.9129702537620084, 0.4591226113143575]], [0.45]]]

raw10 =  [[[[1.198998468541923, 0.611754204266719, 1.7677170617308327]], [[3.4341117624938446, 0.9640180391471003], [-0.05887913748075457, 0.9825724309247841], [2.573053595563823, 0.38627004681749383]], [[1.137932637787607], [2.1164923802567945], [3.232824945210956]], [[3.0946849052258356, 3.1217341494218322, -0.45836455321778186]]], [[[0.5030156034214709]], [[3.9129702537620084, 0.4591226113143575]], [0.45]]]

raws=[raw1, raw2, raw3, raw4, raw5, raw6, raw7, raw8, raw9, raw10]

#raws=[ raw6 ]



In [99]:
#moves
move1={}
move2={}
move3={}
move4={}
move5={}
move6={}
move7={}
move8={}
move9={}

move1={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move2={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move3={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move4={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move5={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move6={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move7={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move8={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move9={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}
move10={'MountainCarContinuousV0': [-0.6, -0.5], 'Action1ws': [-0.4, -0.4],       'OL0C0sm': [-0.55, -0.2], 'OL0C1sm': [0, -0.2], 'OL0C2sm': [0.55, -0.2],     'OL1C0sm': [0, -0.1], 'IV': [-0.8, 0.05], 'IP': [-1.1, 0.5], 'CL1C0': [0, 0.1]}

moves=[move1, move2, move3, move4, move5, move6, move7, move8, move9, move10]

In [100]:
# plotss
plots1 = []
plots2 = []
plots3 = []
plots4 = []
plots5 = []
plots6 = []
plots7 = []
plots8 = []
plots9 = []
plots10 = [
    {'plot_items': {'IP':'ip', 'IV':'iv'},'title':'Inputs'},
    {'plot_items': {'OL0C1sm':'out1'}, 'title':'Output1'},
    {'plot_items': {'OL0C0sm':'out0', 'OL0C2sm':'out2'}, 'title':'Output'},
    {'plot_items': {'Action1ws':'act'}, 'title':'Action'}
    ]

plotss = [plots1, plots2, plots3, plots4, plots5, plots6, plots7, plots8, plots9, plots10]
plotss = [plots10, plots10, plots10, plots10, plots10, plots10, plots10, plots10, plots10, plots10]

In [101]:
def plotting(plots, hpct):
  for plot_item in plots:
      fig = hpct.hierarchy_plots(title=plot_item['title'], plot_items=plot_item['plot_items'])


In [102]:
def run(raws, moves, plot, plotss):
  actionss = []
  for ctr in range(len(raws)):
    raw = raws[ctr]
    move = moves[ctr]
    score, last, hpct = DynamicArchitecture.run_raw(raw=raw, arch_structure=arch_structure, move=move, 
                env=env, runs=runs, inputs=inputs, error_properties=error_properties, inputs_names=inputs_names, 
                summary=False, verbose=verbose, seed=seed, history=history, top_input_indexes=top_inputs,
                error_collector_type=error_collector_type, error_response_type=error_response_type, 
                draw=draw, suffixes=True, error_limit =error_limit )
    print(f'[{ctr+1}] seed {seed} score {score:5.3f} last step {last}')

    if plot:
      plotting(plotss[ctr], hpct)

    actions = hpct.get_history_data()['Action1ws']
    last = actions[-1]
    actions.append(last)

    actionss.append(actions)
  return actionss

In [103]:
env = MountainCarContinuousV0(name='MountainCarContinuousV0', render=render, early_termination=True)

In [104]:
actionss = run(raws, moves, plot, plotss)

Output hidden; open in https://colab.research.google.com to view.

In [105]:
env.close()

In [106]:
#print(actionss)

In [107]:
#actions = hpct.get_history_data()['Action1ws']
#last = actions[-1]
#actions.append(last)


In [108]:
#print(len(actions),actions)

In [109]:
def videos(actionss):
  for actions in actionss:
    genv = gym.make('MountainCarContinuous-v0')
    genv._max_episode_steps = 4000
    env = vid.wrap_env(genv)
    state = env.reset()
    for action in actions:
        #print(action, end = ' ' )
        obs = env.step([action])
        if obs[2]:
          break
        env.render()
    print()
    env.close()
    vid.show_video()

In [110]:
videos(actionss)